##### Copyright 2021 The TensorFlow Authors.

This notebook is adopted from the Tutorial by the TensorFlow authors and modified.

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [ ]:
!pip install -qq roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=" ") # I've hidden my API key
project = rf.workspace().project("plantdoc-ljtfv")
dataset = project.version(8).download("voc")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.1 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to PlantDoc-8 in voc:: 100%|██████████| 8222/8222 [00:02<00:00, 3829.20it/s]


In [ ]:
exit() #restart the notebook

In [ ]:
!pip install -q tflite-model-maker==0.3.4
#!pip install -q tflite-support

Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

There are six steps to training an object detection model:

**Step 1. Choose an object detection model archiecture.**

This tutorial uses the EfficientDet-Lite2 model. EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture. 

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Pixel 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>


In [ ]:
spec = model_spec.get('efficientdet_lite4')

**Step 2. Load the dataset.**

Model Maker will take input data in the CSV format. Use the `ObjectDetectorDataloader.from_csv` method to load the dataset and split them into the training, validation and test images.

* Training images: These images are used to train the object detection model to recognize salad ingredients.
* Validation images: These are images that the model didn't see during the training process. You'll use them to decide when you should stop the training, to avoid [overfitting](https://en.wikipedia.org/wiki/Overfitting).
* Test images: These images are used to evaluate the final model performance.

You can load the CSV file directly from Google Cloud Storage, but you don't need to keep your images on Google Cloud to use Model Maker. You can specify a local CSV file on your computer, and Model Maker will work just fine.

In [ ]:
main_folder = "/content/PlantDoc-8"

In [ ]:
import os
import glob
def get_list(folders):
  path = main_folder + "/"+ folders
  extension = 'xml'
  os.chdir(path)
  result = glob.glob('*.{}'.format(extension))
  result = [i[:-4] for i in result]
  return(result)

In [ ]:
#https://gist.github.com/rotemtam/88d9a4efae243fc77ed4a0f9917c8f6c

import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bbx = member.find('bndbox')
            xmin = int(bbx.find('xmin').text)
            ymin = int(bbx.find('ymin').text)
            xmax = int(bbx.find('xmax').text)
            ymax = int(bbx.find('ymax').text)
            label = member.find('name').text

            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     label,
                     xmin,
                     ymin,
                     xmax,
                     ymax
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


In [ ]:
mapping = xml_to_csv(main_folder + "/train")["class"].unique().tolist()
print(mapping)
print(len(mapping), "classes")


In [ ]:
get_list("train/")[:4]

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(main_folder + "/train", annotations_dir =main_folder + "/train" ,annotation_filenames=get_list("train"), label_map=mapping)
validation_data = object_detector.DataLoader.from_pascal_voc(main_folder + "/valid", annotations_dir =main_folder + "/valid", annotation_filenames=get_list("valid"), label_map=mapping)
test_data = object_detector.DataLoader.from_pascal_voc(main_folder + "/test", annotations_dir = main_folder + "/test",annotation_filenames= get_list("test"), label_map=mapping)

In [ ]:
test_data = object_detector.DataLoader.from_pascal_voc(main_folder + "/test", annotations_dir = main_folder + "/test",annotation_filenames= get_list("test")[:30], label_map=mapping)

**Step 3. Train the TensorFlow model with the training data.**

* The EfficientDet-Lite0 model uses `epochs = 50` by default, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop early to avoid overfitting.
* Set `batch_size = 8` here so you will see that it takes 21 steps to go through the 175 images in the training dataset. 
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data, epochs = 40)

Epoch 1/40
427/427 [==============================] - 634s 1s/step - det_loss: 1.4724 - cls_loss: 1.1711 - box_loss: 0.0060 - reg_l2_loss: 0.1129 - loss: 1.5853 - learning_rate: 0.0090 - gradient_norm: 2.7179 - val_det_loss: 1.0868 - val_cls_loss: 0.8980 - val_box_loss: 0.0038 - val_reg_l2_loss: 0.1132 - val_loss: 1.1999
Epoch 2/40
427/427 [==============================] - 572s 1s/step - det_loss: 0.9807 - cls_loss: 0.7798 - box_loss: 0.0040 - reg_l2_loss: 0.1134 - loss: 1.0941 - learning_rate: 0.0100 - gradient_norm: 2.9909 - val_det_loss: 0.8820 - val_cls_loss: 0.7375 - val_box_loss: 0.0029 - val_reg_l2_loss: 0.1136 - val_loss: 0.9957
Epoch 3/40
427/427 [==============================] - 572s 1s/step - det_loss: 0.8779 - cls_loss: 0.6916 - box_loss: 0.0037 - reg_l2_loss: 0.1138 - loss: 0.9917 - learning_rate: 0.0099 - gradient_norm: 3.0147 - val_det_loss: 0.8878 - val_cls_loss: 0.7697 - val_box_loss: 0.0024 - val_reg_l2_loss: 0.1141 - val_loss: 1.0019
Epoch 4/40
427/427 [===========

**Step 4. Evaluate the model with the test data.**

After training the object detection model using the images in the training dataset, use the remaining 25 images in the test dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the images in the test dataset.

In [ ]:
#model.evaluate(test_data)

**Step 5.  Export as a TensorFlow Lite model.**

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is full integer quantization.

In [ ]:
model.export(export_dir='/content/')

**Step 6.  Evaluate the TensorFlow Lite model.**

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop. 
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('/content/model.tflite', test_data)

30/30 [==============================] - 1450s 48s/step



{'AP': 0.56244385,
 'AP50': 0.69599766,
 'AP75': 0.67652184,
 'AP_/Apple Scab Leaf': 0.7,
 'AP_/Apple leaf': 0.8,
 'AP_/Apple rust leaf': 0.7252475,
 'AP_/Bell_pepper leaf': 0.44125414,
 'AP_/Bell_pepper leaf spot': -1.0,
 'AP_/Blueberry leaf': 0.9,
 'AP_/Cherry leaf': 0.56336635,
 'AP_/Corn Gray leaf spot': -1.0,
 'AP_/Corn leaf blight': 0.9,
 'AP_/Corn rust leaf': -1.0,
 'AP_/Peach leaf': 0.83366334,
 'AP_/Potato leaf': -1.0,
 'AP_/Potato leaf early blight': 0.2539109,
 'AP_/Potato leaf late blight': 0.0,
 'AP_/Raspberry leaf': 0.8582508,
 'AP_/Soyabean leaf': -1.0,
 'AP_/Soybean leaf': -1.0,
 'AP_/Squash Powdery mildew leaf': 0.0,
 'AP_/Strawberry leaf': 0.8442244,
 'AP_/Tomato Early blight leaf': -1.0,
 'AP_/Tomato Septoria leaf spot': -1.0,
 'AP_/Tomato leaf': -1.0,
 'AP_/Tomato leaf bacterial spot': -1.0,
 'AP_/Tomato leaf late blight': 0.7866337,
 'AP_/Tomato leaf mosaic virus': -1.0,
 'AP_/Tomato leaf yellow virus': 0.19219038,
 'AP_/Tomato mold leaf': 0.0,
 'AP_/Tomato two spo

You can download the TensorFlow Lite model file using the left sidebar of Colab. Right-click the `model.tflite` file and choose `Download` to download it to your local computer.

In the next step of the codelab, you'll use the [ObjectDetector API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) of the [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview) to integrate the model into the Android app.